In [1]:
# Load LSTM network and generate text
import sys
import numpy as np
#import mxnet as mx
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import CuDNNLSTM
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
from keras.callbacks import TensorBoard
from keras.utils import np_utils
from keras.utils import multi_gpu_model

Using TensorFlow backend.
C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# load ascii text and covert to lowercase
filename = "pride_and_prejudice.txt"
raw_text = open(filename).read()
raw_text = raw_text + open("hamlet.txt").read()
#raw_text = raw_text.lower()

In [3]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [4]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
print("Characters to int:\n",char_to_int)

Total Characters:  911214
Total Vocab:  86
Characters to int:
 {'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '?': 28, '@': 29, 'A': 30, 'B': 31, 'C': 32, 'D': 33, 'E': 34, 'F': 35, 'G': 36, 'H': 37, 'I': 38, 'J': 39, 'K': 40, 'L': 41, 'M': 42, 'N': 43, 'O': 44, 'P': 45, 'Q': 46, 'R': 47, 'S': 48, 'T': 49, 'U': 50, 'V': 51, 'W': 52, 'X': 53, 'Y': 54, 'Z': 55, '[': 56, ']': 57, '_': 58, 'a': 59, 'b': 60, 'c': 61, 'd': 62, 'e': 63, 'f': 64, 'g': 65, 'h': 66, 'i': 67, 'j': 68, 'k': 69, 'l': 70, 'm': 71, 'n': 72, 'o': 73, 'p': 74, 'q': 75, 'r': 76, 's': 77, 't': 78, 'u': 79, 'v': 80, 'w': 81, 'x': 82, 'y': 83, 'z': 84, '|': 85}


In [5]:
# prepare the dataset of input to output pairs encoded as integers
#seq_length = 30
seq_length = 1
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  911213


In [6]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [7]:
model = Sequential()
model.add(CuDNNLSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [8]:
# define the checkpoint
filepath="weights-improvement-capital-seq1-{epoch:02d}-{loss:.4f}.hdf5"
csv_logger = CSVLogger('training.log')
tensorboard = TensorBoard(log_dir='./logs')
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint,csv_logger,tensorboard]

In [9]:
model.fit(X, y, epochs=20, batch_size=64, callbacks=callbacks_list)

Epoch 1/20
911213/911213 [==============================] - 198s 218us/step - loss: 2.9588

Epoch 00001: loss improved from inf to 2.95882, saving model to weights-improvement-capital-seq1-01-2.9588.hdf5
Epoch 2/20
911213/911213 [==============================] - 194s 213us/step - loss: 2.8418

Epoch 00002: loss improved from 2.95882 to 2.84177, saving model to weights-improvement-capital-seq1-02-2.8418.hdf5
Epoch 3/20
911213/911213 [==============================] - 196s 215us/step - loss: 2.7806

Epoch 00003: loss improved from 2.84177 to 2.78059, saving model to weights-improvement-capital-seq1-03-2.7806.hdf5
Epoch 4/20
911213/911213 [==============================] - 190s 209us/step - loss: 2.7429

Epoch 00004: loss improved from 2.78059 to 2.74290, saving model to weights-improvement-capital-seq1-04-2.7429.hdf5
Epoch 5/20
911213/911213 [==============================] - 190s 209us/step - loss: 2.7180

Epoch 00005: loss improved from 2.74290 to 2.71796, saving model to weights-impr